In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'dev'

In [ ]:
import src.config
import pandas
from deltalake import DeltaTable
from pathlib import Path
import pandas as pd

In [ ]:
# Core logic specific to project/report

import pandas as pd
from pathlib import Path
from deltalake import DeltaTable
import src.config

# def main_report_creation():
# Get Lakehouse tables
df = DeltaTable(src.config.SILVER / "account").to_pandas()
wh_acctloan = DeltaTable(src.config.BRONZE / "wh_acctloan").to_pandas()
wh_loans = DeltaTable(src.config.BRONZE / "wh_loans").to_pandas()

# Need to get rtxn & wh_totalpaymentsdue

# Filter to SBA loans
df = df[df['product'].str.contains('SBA',case=False,na=False)].copy()
df = df[[
    'acctnbr',
    'product',
    'curracctstatcd',
    'ownersortname',
    'noteintrate',
    'Net Balance'
]].copy()
df = df.rename(columns={
    'noteintrate':'Interest Rate (%)',
    'Net Balance':'Closing Balance'
}).copy()

wh_acctloan = wh_acctloan[[
    'acctnbr',
    'currduedate'
]].copy()

wh_acctloan = wh_acctloan.rename(columns={
    'currduedate':'Next Installment Due Date (MM/DD/YYYY)'
}).copy()
wh_acctloan['Next Installment Due Date (MM/DD/YYYY)'] = pd.to_datetime(wh_acctloan['Next Installment Due Date (MM/DD/YYYY)']).copy()
wh_acctloan['acctnbr'] = wh_acctloan['acctnbr'].astype(str)

wh_loans = wh_loans[[
    'acctnbr',
    'intpaidtodate'
]].copy()

wh_loans = wh_loans.rename(columns={
    'intpaidtodate':'Interest Period To (MM/DD/YYYY)'
}).copy()
wh_loans['acctnbr'] = wh_loans['acctnbr'].astype(str)

merged_df = df.merge(wh_acctloan, on='acctnbr', how='left').merge(wh_loans, how='left', on='acctnbr')



In [ ]:
merged_df

In [ ]:
wh_acctloan

In [ ]:
TEMPLATE_PATH = Path(r"C:\Users\w322800\Documents\gh\bcsb-prod\Reports\Credit Loan Review\SBA_Loans\docs\sba loans.xlsx")

In [ ]:
template = pd.read_excel(TEMPLATE_PATH)

In [ ]:
template

In [ ]:
template['Products'].unique()

In [ ]:
TABLE_PATH = src.config.SILVER / "account"

In [ ]:
df = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
df = df[df['product'].str.contains('SBA', case=False, na=False)].copy()

In [ ]:
df

In [ ]:
wh_acctcommon = DeltaTable(src.config.BRONZE / "wh_acctcommon").to_pandas()

In [ ]:
wh_acctcommon

In [ ]:
wh_acctloan = DeltaTable(src.config.BRONZE / "wh_acctloan").to_pandas()

In [ ]:
wh_acctloan

In [ ]:
metadata1 = DeltaTable(src.config.BRONZE / "metadata_lookup_engine1").to_pandas()

In [ ]:
metadata1

In [ ]:
metadata2 = DeltaTable(src.config.BRONZE / "metadata_lookup_engine2").to_pandas()

In [ ]:
metadata2

In [ ]:
import numpy as np

In [ ]:
wh_acctloan['match'] = np.where(wh_acctloan['currduedate'].astype(str) == wh_acctloan['nextduedate'].astype(str),1,0)

In [ ]:
wh_acctloan

In [ ]:
wh_loans = DeltaTable(src.config.BRONZE / "wh_loans").to_pandas()

In [ ]:
wh_loans

In [ ]:
wh_rtxn = DeltaTable(src.config.BRONZE / "wh_rtxn").to_pandas()

In [ ]:
wh_rtxn